In [52]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [53]:
theater_seat_organizer = gp.Model()

# The Parameters

In [95]:
#Uncomment the single file that will be tested

#test_case = "testcase1.txt"
#test_case = "testcase2.txt"
#test_case = "testcase3.txt"
test_case = "testcase4.txt"
#test_case = "testcase5.txt"

#test_name = "Test Case 3 (exceeding theatre capacity)"
#number_of_customers_in_group = [5,7,8,1,4,6,7,8,2,6,15,6,20]
#number_of_seats = 80
#non_vax_cost = 0.01

f = open(test_case,"r")
testfile = f.readlines()

test_name = testfile[0].split("=")[1]

string_num_of_customers_in_group = testfile[1].split(" = ")[1]

#Convert list of string to list of int
number_of_customers_in_group_list = string_num_of_customers_in_group.strip("[").strip("\n").strip("]").split(",")
map_list = map(int, number_of_customers_in_group_list)
number_of_customers_in_group = list(map_list)


number_of_seats = int(testfile[2].split(" = ")[1])

non_vax_cost = float(testfile[3].split(" = ")[1])

# Decision Variables

In [96]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [97]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [98]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [99]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")


In [100]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [ ]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

# Optimize 

In [ ]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2674786 rows, 25302 columns and 5423211 nonzeros
Model fingerprint: 0x7f5f2e4a
Variable types: 0 continuous, 25302 integer (25302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 108.96 (100.11s)


# Post Process

In [92]:
seating_scheme = list()
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
                
                seating_scheme.append((customer_index, group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")
        seating_scheme.append(None)

Seat0: person3 fromGroup1
Seat1: person2 fromGroup1
Seat2: person4 fromGroup1
Seat3: person0 fromGroup1
Seat4: person5 fromGroup1
Seat5: person1 fromGroup1
Seat6: <empty>
Seat7: person1 fromGroup0
Seat8: person3 fromGroup0
Seat9: person4 fromGroup0
Seat10: person2 fromGroup0
Seat11: person0 fromGroup0


In [93]:
genericSpacing = list()
last_group = -1
for value in seating_scheme:
    if(type(value) == tuple and value[1] != last_group):
        genericSpacing.append((None,None))
        genericSpacing.append(value)
        last_group = value[1]
        
    
    elif(type(value) == tuple and value[1] == last_group):
        genericSpacing.append(value)
        
genericSpacing.pop(0)

while(genericSpacing[0][1] == genericSpacing[-1][1]):
    genericSpacing.append(genericSpacing[0])
    genericSpacing.pop(0)
    
genericSpacing.pop(0)
    
for members in genericSpacing:
    if not (members[0] == None):
        print("person " + str(members[0] + 1) + " from group " + str(members[1] + 1))
    else:
        print("<SPACE>")



person 3 from group 2
person 5 from group 2
person 1 from group 2
person 6 from group 2
person 2 from group 2
<SPACE>
person 2 from group 1
person 4 from group 1
person 5 from group 1
person 3 from group 1
person 1 from group 1


In [94]:
for group_num in range(len(groupAccepted)):
    if(groupAccepted[group_num].x != 1):
        print("Group " + str(group_num + 1) + " is excluded")
    else:
        print("Group " + str(group_num + 1) + " is accepted")
#print(groupAccepted)

Group 1 is accepted
Group 2 is accepted
Group 3 is excluded
Group 4 is excluded
Group 5 is excluded
Group 6 is excluded
Group 7 is excluded
